In [132]:
import numpy as np
import pandas as pd
import importlib
import sys
# import keras
from tqdm import tqdm
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from classifiers.cnn import Classifier_CNN
from classifiers.fcn import Classifier_FCN
from classifiers.resnet import Classifier_RESNET
importlib.reload(sys.modules[Classifier_CNN.__module__])
importlib.reload(sys.modules[Classifier_FCN.__module__])
importlib.reload(sys.modules[Classifier_RESNET.__module__])

<module 'classifiers.resnet' from '/home/aasleptsov98/Coursework/cnn_vs_rnn/classifiers/resnet.py'>

In [18]:
app_train = pd.read_csv("input/application_train.csv")
credit_balance = pd.read_csv("input/credit_card_balance.csv")

In [19]:
credit_balance = credit_balance.fillna(0)

In [20]:
ohe = preprocessing.OneHotEncoder()
categories = np.array(list(set(credit_balance["NAME_CONTRACT_STATUS"].astype(str).values))).reshape(-1,1)
ohe.fit(categories)

OneHotEncoder()

In [21]:
encoded = np.array(ohe.transform(credit_balance["NAME_CONTRACT_STATUS"].values.reshape(-1,1)).todense())
temp_df = pd.DataFrame(encoded, columns=range(encoded.shape[1]))

In [22]:
credit_balance = pd.concat([credit_balance.drop(["NAME_CONTRACT_STATUS"], axis=1), temp_df], axis=1)

In [23]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
credit_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,0,1,2,3,4,5,6
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
n_timesteps = 80
n_features = credit_balance.shape[1]

In [81]:
X = []
y = []
for id_cur, df_id_cur in tqdm(credit_balance.groupby("SK_ID_CURR")):
    tmp_df = app_train[app_train["SK_ID_CURR"] == id_cur]
    if len(tmp_df) > 0:
        X.append(df_id_cur[-n_timesteps:].sort_values(by=['MONTHS_BALANCE']).values)
        #y.append(np.array([1, 0]) if tmp_df.iloc[0].TARGET == 0 else np.array([0, 1]))
        y.append(tmp_df.iloc[0].TARGET)

100%|██████████| 103558/103558 [03:02<00:00, 566.87it/s]


In [11]:
lens = []
for data in X:
    lens.append(len(data))

In [11]:
max(lens), min(lens), sum(lens) / len(lens)

(80, 1, 34.55323629250331)

In [82]:
for i in range(len(X)):
    X[i] = np.append(np.zeros((max(n_timesteps - X[i].shape[0], 0), X[i].shape[1]), dtype=int), X[i], axis=0)

In [83]:
X = np.array(X)
y = np.array(y)

In [85]:
zeroes_mask = y == 0
ones_mask = y == 1
zeroes_X = X[zeroes_mask][:7500]
zeroes_y = y[zeroes_mask][:7500]
ones_X = X[ones_mask][:7500]
ones_y = y[ones_mask][:7500]
balanced_X = np.append(zeroes_X, ones_X, axis=0)
balanced_y = np.append(zeroes_y, ones_y, axis=0)

In [61]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1
X_train, X_test, y_train, y_test = train_test_split(balanced_X, balanced_y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [31]:
batch_size = 16

In [45]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(metrics=['accuracy', keras.metrics.AUC()], loss='binary_crossentropy')

In [99]:
model_cnn = Classifier_CNN(X[0].shape, nb_classes=2, verbose=True)

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 80, 29)]          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 74, 6)             1224      
_________________________________________________________________
average_pooling1d_30 (Averag (None, 24, 6)             0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 18, 12)            516       
_________________________________________________________________
average_pooling1d_31 (Averag (None, 6, 12)             0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 72)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)               

In [100]:
model_cnn.fit(X_train, y_train, X_val, y_val)

Epoch 1/2000
750/750 [==============================] - 2s 3ms/step - loss: 0.6895 - accuracy: 0.5397 - val_loss: 0.6923 - val_accuracy: 0.5367
Epoch 2/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6872 - accuracy: 0.5529 - val_loss: 0.6928 - val_accuracy: 0.5413
Epoch 3/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6867 - accuracy: 0.5493 - val_loss: 0.6931 - val_accuracy: 0.5340
Epoch 4/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6861 - accuracy: 0.5518 - val_loss: 0.6952 - val_accuracy: 0.5220
Epoch 5/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6867 - accuracy: 0.5482 - val_loss: 0.6945 - val_accuracy: 0.5340
Epoch 6/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6867 - accuracy: 0.5456 - val_loss: 0.6931 - val_accuracy: 0.5293
Epoch 7/2000
750/750 [==============================] - 2s 2ms/step - loss: 0.6862 - accuracy: 0.5506 - val_loss: 0.6946 - val_accuracy:

KeyboardInterrupt: 

In [112]:
y_pred = model_cnn.predict(X_test)

In [114]:
accuracy_score(y_test, y_pred.round())

0.556

In [110]:
len(y_test)

1500

In [125]:
model_fcn = Classifier_FCN(X[0].shape, nb_classes=2)

Model: "functional_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 80, 29)]          0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 80, 128)           29824     
_________________________________________________________________
batch_normalization_6 (Batch (None, 80, 128)           512       
_________________________________________________________________
activation_6 (Activation)    (None, 80, 128)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 80, 256)           164096    
_________________________________________________________________
batch_normalization_7 (Batch (None, 80, 256)           1024      
_________________________________________________________________
activation_7 (Activation)    (None, 80, 256)         

In [126]:
model_fcn.fit(X_train, y_train, X_val, y_val)

Epoch 1/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.3198 - accuracy: 0.8767 - val_loss: 0.4410 - val_accuracy: 0.7380
Epoch 2/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2764 - accuracy: 0.8960 - val_loss: 0.4392 - val_accuracy: 0.7340
Epoch 3/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2556 - accuracy: 0.9068 - val_loss: 0.3279 - val_accuracy: 0.8920
Epoch 4/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2413 - accuracy: 0.9137 - val_loss: 0.2315 - val_accuracy: 0.9153
Epoch 5/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2506 - accuracy: 0.9059 - val_loss: 0.2316 - val_accuracy: 0.9227
Epoch 6/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2368 - accuracy: 0.9138 - val_loss: 0.2944 - val_accuracy: 0.8953
Epoch 7/2000
750/750 [==============================] - 12s 16ms/step - loss: 0.2333 - accuracy: 0.9140 - val_loss: 0.2573 -

KeyboardInterrupt: 

In [127]:
y_pred = model_fcn.predict(X_test)

In [128]:
accuracy_score(y_test, y_pred.round())

0.9266666666666666

In [136]:
model_resnet = Classifier_RESNET(X[0].shape, nb_classes=2, verbose=True)

Model: "functional_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 80, 29)]     0                                            
__________________________________________________________________________________________________
conv1d_54 (Conv1D)              (None, 80, 64)       14912       input_22[0][0]                   
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 80, 64)       256         conv1d_54[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 80, 64)       0           batch_normalization_21[0][0]     
______________________________________________________________________________________

In [137]:
model_resnet.fit(X_train, y_train, X_val, y_val)

Epoch 1/1500
188/188 [==============================] - 15s 80ms/step - loss: 0.2735 - accuracy: 0.8956 - val_loss: 1.0329 - val_accuracy: 0.6647
Epoch 2/1500
188/188 [==============================] - 15s 78ms/step - loss: 0.2342 - accuracy: 0.9147 - val_loss: 1.1713 - val_accuracy: 0.6333
Epoch 3/1500
188/188 [==============================] - 14s 77ms/step - loss: 0.2143 - accuracy: 0.9231 - val_loss: 0.3710 - val_accuracy: 0.8380
Epoch 4/1500
188/188 [==============================] - 15s 82ms/step - loss: 0.2002 - accuracy: 0.9301 - val_loss: 0.7320 - val_accuracy: 0.7333
Epoch 5/1500
188/188 [==============================] - 16s 83ms/step - loss: 0.2158 - accuracy: 0.9232 - val_loss: 0.2696 - val_accuracy: 0.9067
Epoch 6/1500
188/188 [==============================] - 16s 83ms/step - loss: 0.2067 - accuracy: 0.9268 - val_loss: 0.3695 - val_accuracy: 0.8387
Epoch 7/1500
188/188 [==============================] - 16s 85ms/step - loss: 0.1959 - accuracy: 0.9322 - val_loss: 0.2100 -

KeyboardInterrupt: 

In [138]:
y_pred = model_resnet.predict(X_test)
accuracy_score(y_test, y_pred.round())

0.922